## imports

In [ ]:
from typing import List, Tuple
import numpy as np
import pandas as pd
import re

from scipy.linalg import inv, cholesky
from scipy import stats
from scipy.stats import vonmises, wrapcauchy, norm, cauchy, uniform
from scipy.optimize import minimize
from sklearn.metrics.pairwise import haversine_distances

import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.linear_model import LinearRegression
import math

from tqdm import tqdm

from dataclasses import dataclass

from fiona.crs import from_epsg

import geopandas as gpd
import movingpandas as mpd
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')



CRS_METRIC = from_epsg(4326)


## Plotting AIS DATA

In [386]:
@dataclass
class AISColumnNames:
    Date: str = "timestamp"
    Sampled_Date: str = "sampled_timestamp"
    Latitude: str = "latitude"
    Longitude: str = "longitude"
    Pseudo_Longitude: str = "pseudo_longitude"
    SOG: str = "sog"
    COG: str = "cog"
    Heading: str = "heading"

    n_Latitude: str = "norm_latitude"
    n_Longitude: str = "norm_longitude"
    n_SOG: str = "norm_sog"
    n_COG: str = "norm_cog"
    n_Heading: str = "norm_heading"

    is_synthetic: str = "is_synthetic"
    to_predict: str = "to_predict"

cols: AISColumnNames = AISColumnNames()
target_freq_in_minutes = 10
target_freq: str = f"{target_freq_in_minutes}min"
sample_T: pd.Timedelta = pd.Timedelta(minutes=target_freq_in_minutes)


def get_trajectory_sequences(trajectory_sampled: pd.DataFrame, time_column_name=None
    ) -> List[pd.DataFrame]:
        trajectory_sequences: List[pd.DataFrame] = []  # To store the sequences
        current_sequence = pd.DataFrame(
            columns=trajectory_sampled.columns
        )  # DF To track the current sequence

        # Iterate through the timestamps
        for i in range(len(trajectory_sampled) - 1):
            if (
                trajectory_sampled.index[i + 1]
                - trajectory_sampled.index[i]
                == sample_T
            ):
                # If the difference is 10 minutes, add the current timestamp to the sequence
                if len(current_sequence) == 0:
                    current_sequence = trajectory_sampled.iloc[
                        [i]
                    ]  # Add the first timestamp of the sequence
                current_sequence = pd.concat(
                    [current_sequence, trajectory_sampled.iloc[[i + 1]]],
                    # ignore_index=True,
                )  # Add the next timestamp
            else:
                # If the difference is not 10 minutes, end the current sequence
                if len(current_sequence) != 0:
                    trajectory_sequences.append(
                        current_sequence
                    )  # Store the completed sequence
                    current_sequence = pd.DataFrame(
                        columns=trajectory_sampled.columns
                    )  # Reset the current sequence

        # Handle the last sequence if it ends at the last timestamp
        if len(current_sequence) != 0:
            trajectory_sequences.append(current_sequence)
            
        # handle one last point
        if trajectory_sequences[-1].index[-1] != trajectory_sampled.index[-1]:
            trajectory_sequences.append(trajectory_sampled.iloc[[-1]])
            
        # handle one first point
        if trajectory_sequences[0].index[0] != trajectory_sampled.index[0]:
            trajectory_sequences.append(trajectory_sampled.iloc[[0]])

        return trajectory_sequences
    
def plot_plotly_trajectory_groups(df_groups: List[List[pd.DataFrame]],
                         group_names, 
                         color_sequence=None,
                         line_width=2,
                         marker_size=4):
    if not df_groups:
        raise ValueError("Empty list of DataFrame groups provided")
    
    if color_sequence is None:
        color_sequence = px.colors.qualitative.Plotly
    
    # Create empty figure with proper mapbox setup
    fig = px.scatter_mapbox(lat=[None], lon=[None]).update_layout(
        mapbox_style="open-street-map",
        mapbox_zoom=8,
        height=600
    )
    
    for group_id, df_group in enumerate(df_groups):
        group_color = color_sequence[group_id % len(color_sequence)]
        
        for segment_id, df in enumerate(df_group):
            if len(df) == 0:
                continue  # Skip empty dataframes
                
            customdata = pd.concat([
                pd.Series(df.index, name=cols.Sampled_Date, index=df.index),
                df[cols.SOG],
                df[cols.COG],
                
            ], axis=1)
                        
            # Add line trace for this segment
            fig.add_trace(
                px.line_mapbox(
                    df,
                    lat=cols.Latitude,
                    lon=cols.Longitude,
                    color_discrete_sequence=[group_color]
                ).data[0].update(
                    mode="lines+markers",
                    line=dict(width=line_width),
                    marker=dict(size=marker_size),
                    name=f"{group_names[group_id]}",
                    showlegend=(segment_id == 0),  # Only show legend for first segment
                    legendgroup=f"{group_names[group_id]}",
                    hoverinfo="text",
                    customdata=customdata,
                    hovertemplate=(
                        "Latitude: %{lat}<br>"
                        "Longitude: %{lon}<br>"
                        "Date: %{customdata[0]}<br>"
                        "SOG: %{customdata[1]}<br>"
                        "COG: %{customdata[2]}<br>"
                    )
                )
            )
    
    fig.update_layout(
        margin={"r":0,"t":40,"l":0,"b":0},
        showlegend=True,
        legend_title_text="Trajectory Groups",
        title="Vessel Trajectory"
    )
    
    # Auto-zoom to the data
    if len(df_groups) > 0 and len(df_groups[0]) > 0:
        first_df = df_groups[0][0]
        fig.update_mapboxes(
            center=dict(
                lat=first_df[cols.Latitude].mean(),
                lon=first_df[cols.Longitude].mean()
            )
        )
    
    return fig

## Old AEKF

In [ ]:
def ssa(eps):
    """
    Normalize latitude and longitude residuals to standard range.
    """
    eps[0] = (eps[0] + np.pi/2) % np.pi - np.pi/2
    eps[1] = (eps[1] + np.pi) % (2 * np.pi) - np.pi
    return eps


def AEKF_traj(traj: mpd.Trajectory, alpha: float = 0.2, mmsi_colName="mmsi"):
    """
    Apply Adaptive Extended Kalman Filter to correct maritime vessel trajectory.
    """
    df = traj.df.copy()
    mmsi = df[mmsi_colName].unique()[0]
    
    # WGS-84 ellipsoid constants
    a = 6378137
    f = 1 / 298.257223563
    e = np.sqrt(2 * f - f**2)

    alpha_1, alpha_2 = 0.01, 0.01

    # Initial lat/lon in radians
    lat_rad = np.deg2rad(df[cols.Latitude].values)
    lon_rad = np.deg2rad(df[cols.Longitude].values)

    # Covariances
    Q11, Q22 = 1e8, 1e2
    R11 = R22 = 36
    Qd_base = np.diag([Q11, Q22])
    Rd_base = np.diag([R11, R22])

    P_prd = np.eye(5)
    Cd = np.array([[1, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0]])

    coord_ls, sog, cog = [], [], []
    x_hat = np.array([lat_rad[0], lon_rad[0], 0, 0, 0], dtype=float)

    dt_sec = np.diff(df.index.view(np.int64) // 1_000_000_000)
    dt_min = dt_sec.min()
    dts = np.insert(dt_sec, 0, 0)
    
    try:
        for i in range(len(df)):
            h = dts[i]
            y = np.array([lat_rad[i], lon_rad[i]])

            # Adaptive noise adjustment
            if alpha and h > 0:
                Qd = alpha * (h / dt_min) * Qd_base + (1 - alpha) * Qd_base
                Rd = alpha * (dt_min / h) * Rd_base + (1 - alpha) * Rd_base
            else:
                Qd, Rd = Qd_base, Rd_base

            Ed = h * np.array([[0, 0],
                               [0, 0],
                               [1, 0],
                               [0, 0],
                               [0, 1]])

            Rn = a / np.sqrt(1 - e**2 * np.sin(x_hat[0])**2)
            Rm = Rn * ((1 - e**2) / (1 - e**2 * np.sin(x_hat[0])**2))

            f = np.array([
                (1 / Rm) * x_hat[2] * np.cos(x_hat[3]) * h,
                (1 / (Rn * np.cos(x_hat[0]))) * x_hat[2] * np.sin(x_hat[3]) * h,
                -alpha_1 * x_hat[2],
                x_hat[4] * h,
                -alpha_2 * x_hat[4]
            ])

            A21 = (x_hat[2] * np.sin(x_hat[3]) * np.tan(x_hat[0])) / (Rn * np.cos(x_hat[0]))
            Ad = np.eye(5) + h * np.array([
                [0, 0, (1 / Rm) * np.cos(x_hat[3]), -(1 / Rm) * x_hat[2] * np.sin(x_hat[3]), 0],
                [A21, 0, (1 / (Rn * np.cos(x_hat[0]))) * np.sin(x_hat[3]), (1 / (Rn * np.cos(x_hat[0]))) * x_hat[2] * np.cos(x_hat[3]), 0],
                [0, 0, -alpha_1, 0, 0],
                [0, 0, 0, 0, 1],
                [0, 0, 0, 0, -alpha_2]
            ])

            # print(f)
            x_prd = x_hat + f
            P_hat = Ad @ P_prd @ Ad.T + Ed @ Qd @ Ed.T

            d = ssa(y - Cd @ x_prd)
            S = Cd @ P_hat @ Cd.T + Rd
            K = P_hat @ Cd.T @ np.linalg.inv(S)

            x_hat = x_prd + K @ d
            x_hat[:2] = ssa(x_hat[:2])
            IKC = np.eye(5) - K @ Cd
            P_prd = IKC @ P_hat @ IKC.T + K @ Rd @ K.T

            # Save result
            coord_ls.append(np.rad2deg(x_hat[:2]))
            sog.append(x_hat[2] / (0.51444444 * (h if h > 0 else 1)))  # m/s to knots
            cog.append(np.rad2deg(x_hat[3]) % 360)

        lat_deg, lon_deg = zip(*coord_ls)
        df_corr = pd.DataFrame({cols.Latitude: lat_deg, cols.Longitude: lon_deg, cols.SOG: sog, cols.COG: cog}, index=df.index)
        df_geo = gpd.GeoDataFrame(df_corr, geometry=gpd.points_from_xy(df_corr[cols.Longitude], df_corr[cols.Latitude]), crs=CRS_METRIC)
        return mpd.Trajectory(df_geo, traj_id=mmsi)

    except Exception as e:
        print(f"Error in AEKF_traj: {e}")
        return False

def RMSE_error(collection, new_collections_aekf, mask_traj):
    """
    Calculate RMSE between true and filtered trajectories.
    """
    RMSE_aekf = []
    collection = [collection[i] for i, keep in enumerate(mask_traj) if keep]

    for true_traj, est_traj in tqdm(zip(collection, new_collections_aekf), desc="RMSE"):
        true_coord = np.deg2rad(true_traj.df[[cols.Latitude, cols.Longitude]].values)
        pred_coord = np.deg2rad(est_traj.df[[cols.Latitude, cols.Longitude]].values)
        dist_matrix = haversine_distances(true_coord, pred_coord) * 6371  # km
        diag_dist = np.diag(dist_matrix)
        rmse = np.sqrt((diag_dist**2).mean())
        RMSE_aekf.append(rmse)

    return RMSE_aekf


def AEKF_traj_advanced(traj: pd.DataFrame, alpha: float = 0.2, mmsi_colName="mmsi", max_to_reconstruct=10):
    """
    Apply Adaptive Extended Kalman Filter to correct maritime vessel trajectory.
    """
    df = traj.copy()
    mmsi = df[mmsi_colName].unique()[0]
    
    # WGS-84 ellipsoid constants
    a = 6378137
    f = 1 / 298.257223563
    e = np.sqrt(2 * f - f**2)

    alpha_1, alpha_2 = 0.01, 0.01

    # Initial lat/lon in radians
    lat_rad = np.deg2rad(df[cols.Latitude].values)
    lon_rad = np.deg2rad(df[cols.Longitude].values)

    # Covariances
    Q11, Q22 = 1e8, 1e2
    R11 = R22 = 36
    Qd_base = np.diag([Q11, Q22])
    Rd_base = np.diag([R11, R22])
    delta_q = Qd_base
    delta_r = Rd_base

    P_prd = np.eye(5)
    Cd = np.array([[1, 0, 0, 0, 0],
                   [0, 1, 0, 0, 0]])

    coord_ls, sog, cog, timestamps = [], [], [], []
    x_hat = np.array([lat_rad[0], lon_rad[0], 0, 0, 0], dtype=float)

    dt_sec = np.diff(df.index.view(np.int64) // 1_000_000_000)
    dt_min = dt_sec.min()
    dts = np.insert(dt_sec, 0, 0)
    
    
    try:
        for i in range(len(df)):
            h = dts[i]
            y = np.array([lat_rad[i], lon_rad[i]])
            
            n_steps = max(1, int(h / dt_min))
            
            if n_steps > max_to_reconstruct:
                Qd, Rd = Qd_base, Rd_base
                x_hat = np.array([lat_rad[i], lon_rad[i], 0, 0, 0], dtype=float)
                coord_ls.append(np.rad2deg(x_hat[:2]))
                sog.append(df[cols.SOG].iloc[i])  # m/s to knots
                cog.append(df[cols.COG].iloc[i])
                if len(timestamps) == 0:
                    timestamps.append(df.index[0])
                else:
                    timestamps.append(df.index[i])
                
                continue
            # print(n_steps)

            # estimate next steps even if there is no measurement
            for step in range(n_steps):
                # Adaptive noise adjustment
                if alpha and h > 0:
                    # Qd = (alpha) * Qd_base + (1 - alpha) * delta_q
                    # Rd = (alpha) * Rd_base + (1 - alpha) * delta_r
                    Qd = (alpha) * Qd_base + (1 - alpha) * Qd_base
                    Rd = (alpha) * Rd_base + (1 - alpha) * Rd_base
                else:
                    Qd, Rd = Qd_base, Rd_base

                Ed = h * np.array([[0, 0],
                                [0, 0],
                                [1, 0],
                                [0, 0],
                                [0, 1]])

                Rn = a / np.sqrt(1 - e**2 * np.sin(x_hat[0])**2)
                Rm = Rn * ((1 - e**2) / (1 - e**2 * np.sin(x_hat[0])**2))

                f = np.array([
                    (1 / Rm) * x_hat[2] * np.cos(x_hat[3]) * h,
                    (1 / (Rn * np.cos(x_hat[0]))) * x_hat[2] * np.sin(x_hat[3]) * h,
                    -alpha_1 * x_hat[2],
                    x_hat[4] * h,
                    -alpha_2 * x_hat[4]
                ])

                A21 = (x_hat[2] * np.sin(x_hat[3]) * np.tan(x_hat[0])) / (Rn * np.cos(x_hat[0]))
                Ad = np.eye(5) + h * np.array([
                    [0, 0, (1 / Rm) * np.cos(x_hat[3]), -(1 / Rm) * x_hat[2] * np.sin(x_hat[3]), 0],
                    [A21, 0, (1 / (Rn * np.cos(x_hat[0]))) * np.sin(x_hat[3]), (1 / (Rn * np.cos(x_hat[0]))) * x_hat[2] * np.cos(x_hat[3]), 0],
                    [0, 0, -alpha_1, 0, 0],
                    [0, 0, 0, 0, 1],
                    [0, 0, 0, 0, -alpha_2]
                ])

                # print(f)
                x_prd = x_hat + f
                P_hat = Ad @ P_prd @ Ad.T + Ed @ Qd @ Ed.T

                if step == n_steps - 1:
                    # if there is a corresponding measurement
                    d = ssa(y - Cd @ x_prd)
                    S = Cd @ P_hat @ Cd.T + Rd
                    K = P_hat @ Cd.T @ np.linalg.inv(S)

                    x_hat = x_prd + K @ d
                    x_hat[:2] = ssa(x_hat[:2])
                    IKC = np.eye(5) - K @ Cd
                    P_prd = IKC @ P_hat @ IKC.T + K @ Rd @ K.T
                    
                    # delta_q = (Cd @ K @ np.expand_dims(d, 1) @ np.expand_dims(d, 1).T @ K.T @ Cd.T) * np.eye(2) % 1e8
            
                    # esp = y - Cd @ x_hat
                    # esp = np.expand_dims(esp, 1)
                    # delta_r = (esp @ esp.T + Cd @ P_hat @ Cd.T) *np.eye(2) / R11
                else:
                    pass
                    # x_hat = x_prd
                    # x_hat[:2] = ssa(x_hat[:2])
                    
                # Save result
                coord_ls.append(np.rad2deg(x_hat[:2]))
                sog.append(x_hat[2] / (0.51444444 * (h if h > 0 else 1)))  # m/s to knots
                cog.append(np.rad2deg(x_hat[3]) % 360)
                if len(timestamps) == 0:
                    timestamps.append(df.index[0])
                else:
                    timestamps.append(timestamps[-1] + pd.Timedelta(minutes=dt_min/60))

        lat_deg, lon_deg = zip(*coord_ls)
        df_corr = pd.DataFrame({cols.Latitude: lat_deg, cols.Longitude: lon_deg, cols.SOG: sog, cols.COG: cog}, index=timestamps)
        # df_geo = gpd.GeoDataFrame(df_corr, geometry=gpd.points_from_xy(df_corr.longitude, df_corr.latitude), crs=CRS_METRIC)
        # df_corr_mdp = mpd.Trajectory(df_geo, traj_id=mmsi)
        return df_corr

    except Exception as e:
        print(f"Error in AEKF_traj: {e}")
        return False


In [ ]:
# df = df_original.copy()

# df['target_id'] = mmsi  
# df = df.set_index(cols.Sampled_Date)

# dt_sec = np.diff(df.index.view(np.int64) // 1_000_000_000)
# dt_min = dt_sec.min()
# dts = np.insert(dt_sec, 0, 0)
# # print(dts)

# corrected_df = AEKF_traj_advanced(df, alpha=0.9, mmsi_colName="target_id")
# # corrected_df

# df_sequences = get_trajectory_sequences(df)
# corrected_df_sequences = get_trajectory_sequences(corrected_df)

# fig = plot_plotly_trajectory_groups([corrected_df_sequences, df_sequences], group_names=["AEKF", "Initial trajectory"])
# fig.show()


# gdf = gpd.GeoDataFrame(
#     df,
#     geometry=gpd.points_from_xy(df[cols.Longitude], df[cols.Latitude']),
#     crs="EPSG:4326"  # WGS84, standard for lat/lon
# )

# traj = mpd.Trajectory(gdf, traj_id=mmsi, t='index')

# gdf[cols.Latitude] = pd.to_numeric(gdf[cols.Latitude], errors='coerce')
# gdf[cols.Longitude] = pd.to_numeric(gdf[cols.Longitude], errors='coerce')
# gdf[cols.SOG] = pd.to_numeric(gdf[cols.SOG], errors='coerce')
# gdf[cols.COG] = pd.to_numeric(gdf[cols.COG], errors='coerce')


# corrected_traj = AEKF_traj_advanced(traj, alpha=0.9, mmsi_colName="target_id")

# import movingpandas as mpd
# import hvplot.pandas  # Ensure hvplot is installed

# # Interactive plot with hvplot
# plot = traj.hvplot(
#     geo=True,
#     tiles="OSM",  # Use OpenStreetMap as the basemap
#     c=cols.SOG,      # Color by speed over ground
#     cmap='Viridis',  # Color map
#     line_width=2,
#     title=f"Trajectory for Vessel {traj.id}",
#     colorbar=True
# )
# plot

## NEW AEKF

In [ ]:
class AdaptiveExtendedKalmanFilter:
    """
    Adaptive Extended Kalman Filter for ship trajectory estimation
    
    State vector: [lat, lon, sog, cog, acc, omega]
    - lat: latitude (degrees)
    - lon: longitude (degrees) 
    - sog: speed over ground (knots)
    - cog: course over ground (degrees)
    - acc: acceleration (knots/s)
    - omega: turn rate (degrees/s)
    
    Measurement vector: [lat, lon, sog, cog]
    """
    
    def __init__(self, dt: float = 600.0, alpha_1: float = 0.000, alpha_2: float = 0.000):
        """
        Initialize AEKF
        
        Args:
            dt: Time step in seconds (default 600s = 10 min)
            alpha_1: Acceleration damping coefficient
            alpha_2: Turn rate damping coefficient
        """
        self.dt = dt
        self.alpha_1 = alpha_1
        self.alpha_2 = alpha_2
        
        # WGS-84 ellipsoid constants
        self.a = 6378137.0  # Semi-major axis (m)
        self.f = 1.0 / 298.257223563  # Flattening
        self.e = np.sqrt(2 * self.f - self.f**2)  # Eccentricity
        
        # State dimension
        self.n_states = 6
        self.n_measurements = 4
        
        # Initialize matrices
        self.x = np.zeros(self.n_states)  # State vector
        self.P = np.eye(self.n_states) * 100  # Covariance matrix
        
        # Process noise covariance (Q) - tunable parameters
        self.Q = np.diag([
            1e-8,   # lat process noise (degrees²)
            1e-8,   # lon process noise (degrees²)
            0.01,   # sog process noise (knots²)
            0.1,    # cog process noise (degrees²)
            0.1,    # acc process noise (knots²/s²)
            0.01    # omega process noise (degrees²/s²)
        ])
        
        # Initial measurement noise covariance (R) - will be adapted
        self.R = np.diag([
            1e-6,   # lat measurement noise (degrees²)
            1e-6,   # lon measurement noise (degrees²)
            0.25,   # sog measurement noise (knots²)
            1.0     # cog measurement noise (degrees²)
        ])
        
        # Adaptive parameters
        self.innovation_history = []
        self.max_history = 10
        self.adaptation_factor = 1.0 #0.95
        
    def geodetic_radii(self, lat_deg: float) -> Tuple[float, float]:
        """
        Calculate meridional and prime vertical radii of curvature
        
        Args:
            lat_deg: Latitude in degrees
            
        Returns:
            Rm: Meridional radius of curvature (m)
            Rn: Prime vertical radius of curvature (m)
        """
        lat_rad = np.deg2rad(lat_deg)
        sin_lat = np.sin(lat_rad)
        
        # Prime vertical radius of curvature
        Rn = self.a / np.sqrt(1 - self.e**2 * sin_lat**2)
        
        # Meridional radius of curvature
        Rm = Rn * (1 - self.e**2) / (1 - self.e**2 * sin_lat**2)
        
        return Rm, Rn
    
    def state_transition(self, x: np.ndarray) -> np.ndarray:
        """
        State transition function f(x)
        
        Args:
            x: State vector [lat, lon, sog, cog, acc, omega]
            
        Returns:
            Next state vector
        """
        lat, lon, sog, cog, acc, omega = x
        
        # Convert angles to radians for calculations
        lat_rad = np.deg2rad(lat)
        cog_rad = np.deg2rad(cog)
        
        # Calculate radii of curvature
        Rm, Rn = self.geodetic_radii(lat)
        
        # Convert SOG from knots to m/s for calculations
        sog_ms = sog * 0.514444
        
        # State transition equations
        x_new = np.zeros_like(x)
        
        # Position updates (convert back to degrees)
        x_new[0] = lat + self.dt * np.rad2deg(sog_ms * np.cos(cog_rad) / Rm)
        x_new[1] = lon + self.dt * np.rad2deg(sog_ms * np.sin(cog_rad) / (Rn * np.cos(lat_rad)))
        
        # Speed and course updates
        x_new[2] = sog + self.dt * acc  # SOG in knots
        x_new[3] = cog + self.dt * omega  # COG in degrees
        
        # Acceleration and turn rate updates (damping)
        x_new[4] = acc * (1 - self.alpha_1)
        x_new[5] = omega * (1 - self.alpha_2)
        
        # Normalize COG to [0, 360)
        x_new[3] = x_new[3] % 360
        
        return x_new
    
    def jacobian_f(self, x: np.ndarray) -> np.ndarray:
        """
        Jacobian of state transition function
        
        Args:
            x: State vector
            
        Returns:
            Jacobian matrix F
        """
        lat, lon, sog, cog, acc, omega = x
        
        lat_rad = np.deg2rad(lat)
        cog_rad = np.deg2rad(cog)
        
        Rm, Rn = self.geodetic_radii(lat)
        sog_ms = sog * 0.514444
        
        cos_lat = np.cos(lat_rad)
        sin_lat = np.sin(lat_rad)
        cos_cog = np.cos(cog_rad)
        sin_cog = np.sin(cog_rad)
        
        F = np.eye(self.n_states)
        
        # Partial derivatives for latitude equation
        # ∂lat_new/∂lat (includes curvature dependency)
        dRm_dlat = self.a * (1 - self.e**2) * 2 * self.e**2 * sin_lat * cos_lat / \
                   ((1 - self.e**2 * sin_lat**2)**1.5 * (1 - self.e**2 * sin_lat**2))
        F[0, 0] = 1 - self.dt * np.rad2deg(sog_ms * cos_cog * dRm_dlat / (Rm**2))
        
        # ∂lat_new/∂sog
        F[0, 2] = self.dt * np.rad2deg(0.514444 * cos_cog / Rm)
        
        # ∂lat_new/∂cog
        F[0, 3] = -self.dt * np.rad2deg(sog_ms * sin_cog / Rm) * np.pi/180
        
        # Partial derivatives for longitude equation
        # ∂lon_new/∂lat
        dRn_dlat = -self.a * self.e**2 * sin_lat * cos_lat / (1 - self.e**2 * sin_lat**2)**1.5
        F[1, 0] = self.dt * np.rad2deg(sog_ms * sin_cog * sin_lat / (Rn * cos_lat**2)) * np.pi/180 - \
                  self.dt * np.rad2deg(sog_ms * sin_cog * dRn_dlat / (Rn**2 * cos_lat)) * np.pi/180
        
        # ∂lon_new/∂sog
        F[1, 2] = self.dt * np.rad2deg(0.514444 * sin_cog / (Rn * cos_lat))
        
        # ∂lon_new/∂cog
        F[1, 3] = self.dt * np.rad2deg(sog_ms * cos_cog / (Rn * cos_lat)) * np.pi/180
        
        # SOG equation partials
        F[2, 2] = 1  # ∂sog_new/∂sog
        F[2, 4] = self.dt  # ∂sog_new/∂acc
        
        # COG equation partials
        F[3, 3] = 1  # ∂cog_new/∂cog
        F[3, 5] = self.dt  # ∂cog_new/∂omega
        
        # Acceleration damping
        F[4, 4] = 1 - self.alpha_1 * self.dt
        
        # Turn rate damping
        F[5, 5] = 1 - self.alpha_2 * self.dt
        
        return F
    
    def measurement_function(self, x: np.ndarray) -> np.ndarray:
        """
        Measurement function h(x)
        
        Args:
            x: State vector
            
        Returns:
            Predicted measurement vector
        """
        return x[:4]  # Measure [lat, lon, sog, cog]
    
    def jacobian_h(self, x: np.ndarray) -> np.ndarray:
        """
        Jacobian of measurement function
        
        Args:
            x: State vector
            
        Returns:
            Jacobian matrix H
        """
        H = np.zeros((self.n_measurements, self.n_states))
        H[:4, :4] = np.eye(4)  # Direct measurement of first 4 states
        return H
    
    def adapt_noise_covariance(self, innovation: np.ndarray, S: np.ndarray):
        """
        Adapt measurement noise covariance based on innovation
        
        Args:
            innovation: Innovation vector
            S: Innovation covariance matrix
        """
        # Store innovation for adaptation
        self.innovation_history.append(innovation)
        if len(self.innovation_history) > self.max_history:
            self.innovation_history.pop(0)
        
        if len(self.innovation_history) >= 3:
            # Calculate sample covariance of innovations
            innovations = np.array(self.innovation_history)
            sample_cov = np.cov(innovations.T)
            
            # Adaptive update of R
            self.R = self.adaptation_factor * self.R + \
                     (1 - self.adaptation_factor) * (sample_cov - 
                     self.jacobian_h(self.x) @ self.P @ self.jacobian_h(self.x).T)
            
            # Ensure R remains positive definite
            try:
                cholesky(self.R)
            except np.linalg.LinAlgError:
                self.R = self.R + np.eye(self.n_measurements) * 1e-6
    
    def predict(self):
        """Prediction step"""
        # State prediction
        self.x = self.state_transition(self.x)
        
        # Covariance prediction
        F = self.jacobian_f(self.x)
        self.P = F @ self.P @ F.T + self.Q
    
    def update(self, z: np.ndarray):
        """
        Update step
        
        Args:
            z: Measurement vector [lat, lon, sog, cog]
        """
        # Predicted measurement
        h = self.measurement_function(self.x)
        
        # Innovation
        y = z - h
        
        # Handle COG wraparound in innovation
        if y[3] > 180:
            y[3] -= 360
        elif y[3] < -180:
            y[3] += 360
        
        # Measurement Jacobian
        H = self.jacobian_h(self.x)
        
        # Innovation covariance
        S = H @ self.P @ H.T + self.R
        
        # Adapt noise covariance
        self.adapt_noise_covariance(y, S)
        
        # Kalman gain
        K = self.P @ H.T @ inv(S)
        
        # State update
        self.x = self.x + K @ y
        
        # Covariance update (Joseph form for numerical stability)
        I = np.eye(self.n_states)
        IKH = I - K @ H
        self.P = IKH @ self.P @ IKH.T + K @ self.R @ K.T
        
        # Normalize COG
        self.x[3] = self.x[3] % 360
    
    def initialize(self, z0: np.ndarray, dt: float):
        """
        Initialize filter with first measurement
        
        Args:
            z0: Initial measurement [lat, lon, sog, cog]
            dt: Time step
        """
        self.dt = dt
        self.x[:4] = z0
        self.x[4:] = 0  # Initialize acc and omega to zero
        
        # Initial covariance - higher uncertainty for unobserved states
        self.P = np.diag([1e-6, 1e-6, 0.25, 1.0, 1.0, 0.1])

def process_trajectory_data(df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    """
    Process trajectory DataFrame for AEKF
    
    Args:
        df: DataFrame with columns [latitude, longitude, sog, cog] and timestamp index
        
    Returns:
        measurements: Array of measurements
        time_steps: Array of time steps in seconds
    """
    # Calculate time steps
    timestamps = df.index
    time_steps = np.array([(timestamps[i] - timestamps[i-1]).total_seconds() 
                          for i in range(1, len(timestamps))])
    
    # Extract measurements
    measurements = df[[cols.Latitude, cols.Longitude, cols.SOG, cols.COG]].values
    
    return measurements, time_steps

# Example usage
def run_aekf_example(df: pd.DataFrame, max_to_predict: int = 10):
    """Example of running AEKF on sample data"""   
    # Process data
    measurements, time_steps = process_trajectory_data(df)
    
    # Initialize AEKF
    aekf = AdaptiveExtendedKalmanFilter()
    
    # Initialize with first measurement
    aekf.dt = time_steps.min()
    aekf.initialize(measurements[0], aekf.dt)
    
    # Process measurements
    states = [aekf.x.copy()]
    timestamps = [df.index[0]]
    covariances = [np.diag(aekf.P).copy()]
    
    for i in range(1, len(measurements)):
        # Set time step
        # aekf.dt = time_steps[i-1]
        n_steps = int(time_steps[i-1]/aekf.dt)
        
        if n_steps > max_to_predict:
            aekf.initialize(measurements[i], aekf.dt)
            states.append(aekf.x.copy())
            timestamps.append(df.index[i])
            continue
            
        for step in range(n_steps):
            # Predict
            aekf.predict()
            if step != n_steps - 1:
                states.append(aekf.x.copy())
                timestamps.append(timestamps[-1] + pd.Timedelta(minutes=aekf.dt/60))
        
        # Update with measurement
        aekf.update(measurements[i])
        
        # Store results
        states.append(aekf.x.copy())
        covariances.append(np.diag(aekf.P).copy())
        timestamps.append(timestamps[-1] + pd.Timedelta(minutes=aekf.dt/60))
    
    states = np.array(states)
    covariances = np.array(covariances)
    
    # Print results
    state_names = [cols.Latitude, cols.Longitude, cols.SOG, cols.COG, 'Acceleration', 'Turn Rate']
    units = ['deg', 'deg', 'knots', 'deg', 'knots/s', 'deg/s']
    
    print("AEKF Results:")
    print("=" * 60)
    for i, (name, unit) in enumerate(zip(state_names, units)):
        print(f"{name:12} | Final: {states[-1, i]:10.6f} {unit:8} | "
              f"Std: {np.sqrt(covariances[-1, i]):8.6f}")
    
    states_df = pd.DataFrame(states, columns=state_names)
    states_df.index = timestamps
    
    return states_df, covariances, df

def RMSE_error_new(collection, new_collections_aekf, mask_traj):
    """
    Calculate RMSE between true and filtered trajectories.
    """
    RMSE_aekf = []
    collection = [collection[i] for i, keep in enumerate(mask_traj) if keep]

    for true_traj, est_traj in tqdm(zip(collection, new_collections_aekf), desc="RMSE"):
        true_coord = np.deg2rad(true_traj.df[[cols.Latitude, cols.Longitude]].values)
        pred_coord = np.deg2rad(est_traj.df[[cols.Latitude, cols.Longitude]].values)
        dist_matrix = haversine_distances(true_coord, pred_coord) * 6371  # km
        diag_dist = np.diag(dist_matrix)
        rmse = np.sqrt((diag_dist**2).mean())
        RMSE_aekf.append(rmse)

    return RMSE_aekf

## Old Angles

In [ ]:

# def fit_circular_distributions(angles):
#     """
#     Takes a pandas Series of angles (degrees, -180 to 180) and evaluates the likelihood
#     of various circular distributions (von Mises, wrapped normal, wrapped Cauchy, uniform).
    
#     Parameters:
#     angles (pd.Series): Series of angles in degrees (-180 to 180).
    
#     Returns:
#     dict: Dictionary with distribution names, log-likelihoods, and fitted parameters.
#     """
#     # Convert angles to radians
#     angles_rad = np.deg2rad(angles)
    
#     # angles are in [-pi, pi)
#     # angles_rad = np.mod(angles_rad, np.pi)
    
#     # Initialize results dictionary
#     results = {}
    
#     # 1. Von Mises Distribution
#     def neg_log_likelihood_vonmises(params, data):
#         mu, kappa = params
#         return -np.sum(vonmises.logpdf(data, kappa=kappa, loc=mu))
    
#     # Initial guess: mean angle and kappa=1
#     mu_init = np.angle(np.mean(np.exp(1j * angles_rad)))  # Circular mean
#     kappa_init = 1.0
#     res_vonmises = minimize(
#         neg_log_likelihood_vonmises,
#         x0=[mu_init, kappa_init],
#         args=(angles_rad,),
#         bounds=[(-np.pi, np.pi), (0, 100)]  # kappa > 0, mu in [-pi, pi]
#     )
#     results['von_mises'] = {
#         'log_likelihood': -res_vonmises.fun,
#         'parameters': {'mu': res_vonmises.x[0], 'kappa': res_vonmises.x[1]}
#     }
    
#     # 2. Wrapped Normal Distribution
#     def wrapped_normal_logpdf(data, mu, sigma):
#         # Approximate wrapped normal by summing over a few wraps
#         n = 10  # Number of wraps to consider
#         logpdf = np.zeros_like(data)
#         for k in range(-n, n+1):
#             logpdf += norm.pdf(data + 2 * np.pi * k, loc=mu, scale=sigma)
#         return np.log(np.maximum(logpdf, 1e-10))  # Avoid log(0)
    
#     def neg_log_likelihood_wrapped_normal(params, data):
#         mu, sigma = params
#         return -np.sum(wrapped_normal_logpdf(data, mu, sigma))
    
#     sigma_init = np.std(angles_rad)  # Initial guess for sigma
#     res_wrapped_normal = minimize(
#         neg_log_likelihood_wrapped_normal,
#         x0=[mu_init, sigma_init],
#         args=(angles_rad,),
#         bounds=[(-np.pi, np.pi), (0.01, 10)]  # sigma > 0
#     )
#     results['wrapped_normal'] = {
#         'log_likelihood': -res_wrapped_normal.fun,
#         'parameters': {'mu': res_wrapped_normal.x[0], 'sigma': res_wrapped_normal.x[1]}
#     }
    
#     # 3. Wrapped Cauchy Distribution
#     def wrapped_cauchy_logpdf(data, mu, gamma):
#         # Approximate wrapped Cauchy
#         n = 10
#         logpdf = np.zeros_like(data)
#         for k in range(-n, n+1):
#             logpdf += cauchy.pdf(data + 2 * np.pi * k, loc=mu, scale=gamma)
#         return np.log(np.maximum(logpdf, 1e-10))
    
#     def neg_log_likelihood_wrapped_cauchy(params, data):
#         mu, gamma = params
#         return -np.sum(wrapped_cauchy_logpdf(data, mu, gamma))
    
#     gamma_init = 0.1  # Initial guess for scale
#     res_wrapped_cauchy = minimize(
#         neg_log_likelihood_wrapped_cauchy,
#         x0=[mu_init, gamma_init],
#         args=(angles_rad,),
#         bounds=[(-np.pi, np.pi), (0.01, 5)]  # gamma > 0
#     )
#     results['wrapped_cauchy'] = {
#         'log_likelihood': -res_wrapped_cauchy.fun,
#         'parameters': {'mu': res_wrapped_cauchy.x[0], 'gamma': res_wrapped_cauchy.x[1]}
#     }
    
#     # 4. Uniform Circular Distribution
#     # Log-likelihood for uniform is constant: log(1/(2pi)) per sample
#     uniform_logpdf = -np.log(2 * np.pi)
#     results['uniform'] = {
#         'log_likelihood': uniform_logpdf * len(angles_rad),
#         'parameters': {}  # No parameters to estimate
#     }
    
#     return results

# def wrapped_normal_pdf(x, mu, sigma, terms=5):
#     """
#     Evaluate the wrapped normal PDF at angle x.
#     Parameters:
#         x     : array of angles in radians (0 to 2π)
#         mu    : mean direction
#         sigma : standard deviation of underlying normal
#         terms : number of terms to sum on each side
#     """
#     k = np.arange(-terms, terms+1)
#     x = np.atleast_1d(x)
#     shifted = x[:, None] - mu + 2 * np.pi * k  # shape (N, 2*terms+1)
#     gaussians = np.exp(-0.5 * (shifted / sigma)**2)
#     norm = sigma * np.sqrt(2 * np.pi)
#     return np.sum(gaussians, axis=1) / norm

# def neg_log_likelihood_wrapped_normal(params, data):
#     mu, log_sigma = params
#     sigma = np.exp(log_sigma)  # optimize log(sigma) for stability
#     pdf_vals = wrapped_normal_pdf(data, mu, sigma, terms=5)
#     pdf_vals = np.clip(pdf_vals, 1e-12, None)  # prevent log(0)
#     return -np.sum(np.log(pdf_vals))

# def fit_wrapped_normal(data):
#     """
#     Fit wrapped normal using MLE.
#     Returns: mu, sigma
#     """
#     data = np.mod(data, 2*np.pi)  # ensure [0, 2π]
#     mu_init = np.angle(np.mean(np.exp(1j * data)))
#     sigma_init = np.std(data)
#     res = minimize(neg_log_likelihood_wrapped_normal, 
#                    x0=[mu_init, np.log(sigma_init)],
#                    args=(data,),
#                    bounds=[(0, 2*np.pi), (np.log(1e-3), np.log(10))])
#     mu, log_sigma = res.x
#     sigma = np.exp(log_sigma)
#     log_lik = -res.fun
#     return mu, sigma, log_lik

# def evaluate_circular_distributions(angle_series: pd.Series) -> pd.DataFrame:
#     # Convert to radians
#     data = np.deg2rad(angle_series.dropna().values)
#     print(data)
#     data = np.mod(data, 2*np.pi)
#     print(data)
    
#     # Define distributions
#     results = []
    
#     # 1. Von Mises
#     kappa, loc, scale = vonmises.fit(data, fscale=1)  # scale fixed to 1 for circular data
#     log_lik = np.sum(vonmises.logpdf(data, kappa, loc=loc))
#     aic = 2 * 2 - 2 * log_lik
#     results.append(('vonmises', log_lik, aic))
    
#     # 2. Wrapped Cauchy
#     fit_params = wrapcauchy.fit(data, floc=0, fscale=2*np.pi)
#     c, loc, scale = fit_params
#     log_lik = np.sum(wrapcauchy.logpdf(data, c, loc=loc, scale=scale))
#     aic = 2 * 1 - 2 * log_lik  # 1 parameter estimated: c
#     results.append(('wrapcauchy', log_lik, aic))
    
#     # 3. Uniform
#     log_lik = np.sum(uniform.logpdf(data, loc=0, scale=2*np.pi))
#     aic = 2 * 0 - 2 * log_lik
#     results.append(('uniform', log_lik, aic))
    

#     # 3.  Wrapped Normal
#     mu, sigma, log_lik = fit_wrapped_normal(data)
#     aic = 2 * 2 - 2 * log_lik  # 2 parameters: mu, sigma
#     results.append(('wrapped_normal', log_lik, aic))

#     # Normalize AIC scores to get probabilities
#     aics = np.array([r[2] for r in results])
#     delta_aic = aics - np.min(aics)
#     rel_likelihoods = np.exp(-0.5 * delta_aic)
#     probabilities = rel_likelihoods / np.sum(rel_likelihoods)

#     # Prepare output
#     return pd.DataFrame({
#         'distribution': [r[0] for r in results],
#         'log_likelihood': [r[1] for r in results],
#         'AIC': aics,
#         'probability': probabilities
#     }).sort_values(by='probability', ascending=False)

## Angles

In [ ]:
def calculate_turning_angles(df):
    """
    Calculate turning angles between consecutive points in a DataFrame using latitude and longitude.
    
    Parameters:
    df (pd.DataFrame): DataFrame with timestamp index and 'latitude' and 'longitude' columns (in degrees)
    
    Returns:
    pd.Series: Series with turning angles in degrees, indexed by the second timestamp of each pair
    """
    def calculate_bearing(lat1, lon1, lat2, lon2):
        """
        Calculate the bearing (azimuth) between two points in degrees.
        """
        # Convert latitude and longitude to radians
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        
        # Difference in longitude
        dlon = lon2 - lon1
        
        # Calculate bearing
        x = np.sin(dlon) * np.cos(lat2)
        y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
        bearing = np.arctan2(x, y)
        
        # Convert to degrees and normalize to [0, 360]
        bearing = np.degrees(bearing)
        bearing = (bearing + 360) % 360
        return bearing
    
    # Initialize lists to store bearings and timestamps
    bearings = []
    valid_indices = []
    
    # Iterate over consecutive pairs of points
    for i in range(len(df) - 1):
        lat1, lon1 = df.iloc[i]['latitude'], df.iloc[i]['longitude']
        lat2, lon2 = df.iloc[i + 1]['latitude'], df.iloc[i + 1]['longitude']
        bearing = calculate_bearing(lat1, lon1, lat2, lon2)
        bearings.append(bearing)
        valid_indices.append(df.index[i + 1])
    
    # Calculate turning angles as the difference between consecutive bearings
    turning_angles = []
    for i in range(len(bearings) - 1):
        angle = bearings[i + 1] - bearings[i]
        # Normalize to [-180, 180]
        angle = ((angle + 180) % 360 - 180)
        turning_angles.append(angle)
    
    # Create Series with turning angles, indexed by the second timestamp of each triplet
    result = pd.Series(turning_angles, index=valid_indices[1:], name='turning_angle')
    
    return result


def angles_rad_to_unique_bins(angles_rad, num_bins=72):
    some_bins = np.linspace(0, 2 * np.pi, num_bins + 1)
    some_bin_indices = np.digitize(angles_rad, some_bins, right=False)
    some_bin_indices = np.clip(some_bin_indices - 1, 0, num_bins - 1)
    some_bin_midpoints = (some_bins[:-1] + some_bins[1:]) / 2
    binned_angles = some_bin_midpoints[some_bin_indices]
    angles_rad = np.unique(binned_angles)
    return angles_rad

def plot_circular_distribution(angles, bins=36, title="Circular Distribution of Angles", use_bins=True):
    """
    Create a polar histogram for a pandas Series of angles (in degrees) using Plotly.
    
    Parameters:
    angles (pd.Series): Series containing angles in degrees
    bins (int): Number of bins for the histogram (default: 36)
    title (str): Plot title (default: "Circular Distribution of Angles")
    
    Returns:
    plotly.graph_objects.Figure: The created figure object
    """
    # Convert angles to radians
    angles_rad = np.deg2rad(angles)
    
    if use_bins:
        angles_rad= angles_rad_to_unique_bins(angles_rad)
    
    # Create histogram
    counts, bin_edges = np.histogram(angles_rad, bins=bins, range=(-np.pi, np.pi))
    # Calculate bin centers in degrees for plotting
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    bin_centers_deg = np.rad2deg(bin_centers)
    
    # Create polar bar plot
    fig = go.Figure()
    
    fig.add_trace(
        go.Barpolar(
            r=counts,
            theta=bin_centers_deg,
            width=360/bins,  # Width of each bar in degrees
            marker=dict(
                color=counts,
                colorscale='Aggrnyl',
                showscale=True,
                colorbar_title="Count"
            ),
            name='Angle Distribution'
        )
    )
    
    # Update layout for better visualization
    fig.update_layout(
        title=title,
        polar=dict(
            angularaxis=dict(
                rotation=90,  # Rotate so 0° is at top
                direction="clockwise",
                tickvals=np.linspace(0, 360, 8, endpoint=False),
                ticktext=[f"{int(t)}°" for t in np.linspace(0, 360, 8, endpoint=False)]
            ),
            # radialaxis=dict(
            #     visible=True,
            #     title="Count"
            # )
        ),
        showlegend=False
    )
    
    return fig

def fit_circular_distributions(angles_series):
    """
    Test a pandas Series of angles (-180 to 180) against various circular distributions
    and return the probability/likelihood of each distribution fitting the data.
    
    Parameters:
    -----------
    angles_series : pd.Series
        Series of angles in degrees from -180 to 180
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with distribution names, parameters, log-likelihood, AIC, BIC, and relative probabilities
    """
    
    # Convert angles to radians and ensure they're in [0, 2π)
    angles_rad = np.deg2rad(angles_series.dropna())
    angles_rad = (angles_rad + 2*np.pi) % (2*np.pi)
    
    num_bins = 72
    bins = np.linspace(0, 2 * np.pi, num_bins + 1)
    bin_indices = np.digitize(angles_rad, bins, right=False)
    bin_indices = np.clip(bin_indices - 1, 0, num_bins - 1)
    bin_midpoints = (bins[:-1] + bins[1:]) / 2
    binned_angles = bin_midpoints[bin_indices]
    angles_rad = np.unique(binned_angles)
    
    n = len(angles_rad)
    
    if n < 2:
        raise ValueError("Need at least 2 data points")
    
    results = []
    
    # 1. Uniform Circular Distribution
    def uniform_circular_loglik(angles):
        return n * np.log(1/(2*np.pi))
    
    ll_uniform = uniform_circular_loglik(angles_rad)
    results.append({
        'distribution': 'Uniform Circular',
        'parameters': {},
        'log_likelihood': ll_uniform,
        'n_params': 0
    })
    
    # 2. Von Mises Distribution
    def vonmises_negloglik(params, angles):
        mu, kappa = params
        if kappa <= 0:
            return np.inf
        try:
            # Von Mises log-likelihood
            ll = n * np.log(1/(2*np.pi*stats.i0(kappa))) + kappa * np.sum(np.cos(angles - mu))
            return -ll
        except:
            return np.inf
    
    # Fit Von Mises
    try:
        # Initial estimates
        mean_angle = np.arctan2(np.mean(np.sin(angles_rad)), np.mean(np.cos(angles_rad)))
        R = np.sqrt(np.mean(np.cos(angles_rad))**2 + np.mean(np.sin(angles_rad))**2)
        kappa_init = max(0.1, 2*R/(1-R**2)) if R < 0.95 else 10
        
        res = minimize(vonmises_negloglik, [mean_angle, kappa_init], 
                      args=(angles_rad,), method='L-BFGS-B',
                      bounds=[(-np.pi, np.pi), (0.01, 100)])
        
        if res.success:
            mu_fit, kappa_fit = res.x
            ll_vonmises = -res.fun
            results.append({
                'distribution': 'Von Mises',
                'parameters': {'mu': np.rad2deg(mu_fit), 'kappa': kappa_fit},
                'log_likelihood': ll_vonmises,
                'n_params': 2
            })
    except:
        pass
    
    # 3. Wrapped Normal Distribution
    def wrapped_normal_negloglik(params, angles):
        mu, sigma = params
        if sigma <= 0:
            return np.inf
        try:
            # Wrapped normal approximation using first few terms
            ll = 0
            for angle in angles:
                density = 0
                for k in range(-3, 4):  # Sum over wrapping terms
                    density += np.exp(-((angle - mu + 2*np.pi*k)**2)/(2*sigma**2))
                density /= (sigma * np.sqrt(2*np.pi))
                ll += np.log(max(density, 1e-300))
            return -ll
        except:
            return np.inf
    
    # Fit Wrapped Normal
    try:
        mean_angle = np.arctan2(np.mean(np.sin(angles_rad)), np.mean(np.cos(angles_rad)))
        sigma_init = 1.0
        
        res = minimize(wrapped_normal_negloglik, [mean_angle, sigma_init],
                      args=(angles_rad,), method='L-BFGS-B',
                      bounds=[(-np.pi, np.pi), (0.1, 5)])
        
        if res.success:
            mu_fit, sigma_fit = res.x
            ll_wrapped_normal = -res.fun
            results.append({
                'distribution': 'Wrapped Normal',
                'parameters': {'mu': np.rad2deg(mu_fit), 'sigma': sigma_fit},
                'log_likelihood': ll_wrapped_normal,
                'n_params': 2
            })
    except:
        pass
    
    # 4. Wrapped Gaussian Mixture (2 components)
    def wrapped_gaussian_mixture_negloglik(params, angles):
        if len(params) != 5:  # mu1, sigma1, mu2, sigma2, weight1
            return np.inf
        
        mu1, sigma1, mu2, sigma2, w1 = params
        w2 = 1 - w1
        
        if sigma1 <= 0 or sigma2 <= 0 or not (0 < w1 < 1):
            return np.inf
        
        try:
            ll = 0
            for angle in angles:
                # Component 1
                density1 = 0
                for k in range(-3, 4):
                    density1 += np.exp(-((angle - mu1 + 2*np.pi*k)**2)/(2*sigma1**2))
                density1 /= (sigma1 * np.sqrt(2*np.pi))
                
                # Component 2
                density2 = 0
                for k in range(-3, 4):
                    density2 += np.exp(-((angle - mu2 + 2*np.pi*k)**2)/(2*sigma2**2))
                density2 /= (sigma2 * np.sqrt(2*np.pi))
                
                # Mixture
                mixture_density = w1 * density1 + w2 * density2
                ll += np.log(max(mixture_density, 1e-300))
            
            return -ll
        except:
            return np.inf
    
    # Fit Wrapped Gaussian Mixture (2 components)
    try:
        # Initialize with k-means-like approach on angles
        # Simple initialization: split data into two groups
        sorted_angles = np.sort(angles_rad)
        mid_idx = len(sorted_angles) // 2
        
        mu1_init = np.arctan2(np.mean(np.sin(sorted_angles[:mid_idx])), 
                             np.mean(np.cos(sorted_angles[:mid_idx])))
        mu2_init = np.arctan2(np.mean(np.sin(sorted_angles[mid_idx:])), 
                             np.mean(np.cos(sorted_angles[mid_idx:])))
        
        # Ensure means are separated
        if abs(mu1_init - mu2_init) < np.pi/4:
            mu2_init = mu1_init + np.pi
            mu2_init = (mu2_init + np.pi) % (2*np.pi) - np.pi
        
        sigma1_init = 1.0
        sigma2_init = 1.0
        w1_init = 0.5
        
        res = minimize(wrapped_gaussian_mixture_negloglik, 
                      [mu1_init, sigma1_init, mu2_init, sigma2_init, w1_init],
                      args=(angles_rad,), method='L-BFGS-B',
                      bounds=[(-np.pi, np.pi), (0.1, 5), (-np.pi, np.pi), (0.1, 5), (0.1, 0.9)])
        
        if res.success:
            mu1_fit, sigma1_fit, mu2_fit, sigma2_fit, w1_fit = res.x
            ll_wgm = -res.fun
            results.append({
                'distribution': 'Wrapped Gaussian Mixture (2 comp)',
                'parameters': {
                    'mu1': np.rad2deg(mu1_fit), 'sigma1': sigma1_fit,
                    'mu2': np.rad2deg(mu2_fit), 'sigma2': sigma2_fit,
                    'weight1': w1_fit, 'weight2': 1-w1_fit
                },
                'log_likelihood': ll_wgm,
                'n_params': 5
            })
    except:
        pass
    
    # 5. Wrapped Cauchy Distribution
    def wrapped_cauchy_negloglik(params, angles):
        mu, rho = params
        if not (0 < rho < 1):
            return np.inf
        try:
            ll = n * np.log((1-rho**2)/(2*np.pi)) - np.sum(np.log(1 + rho**2 - 2*rho*np.cos(angles - mu)))
            return -ll
        except:
            return np.inf
    
    # Fit Wrapped Cauchy
    try:
        mean_angle = np.arctan2(np.mean(np.sin(angles_rad)), np.mean(np.cos(angles_rad)))
        rho_init = 0.5
        
        res = minimize(wrapped_cauchy_negloglik, [mean_angle, rho_init],
                      args=(angles_rad,), method='L-BFGS-B',
                      bounds=[(-np.pi, np.pi), (0.01, 0.99)])
        
        if res.success:
            mu_fit, rho_fit = res.x
            ll_wrapped_cauchy = -res.fun
            results.append({
                'distribution': 'Wrapped Cauchy',
                'parameters': {'mu': np.rad2deg(mu_fit), 'rho': rho_fit},
                'log_likelihood': ll_wrapped_cauchy,
                'n_params': 2
            })
    except:
        pass
    
    # Calculate AIC, BIC, and relative probabilities
    df = pd.DataFrame(results)
    if len(df) == 0:
        return pd.DataFrame()
    
    df['AIC'] = 2 * df['n_params'] - 2 * df['log_likelihood']
    df['BIC'] = np.log(n) * df['n_params'] - 2 * df['log_likelihood']
    
    # Calculate Akaike weights (relative probabilities based on AIC)
    min_aic = df['AIC'].min()
    df['delta_AIC'] = df['AIC'] - min_aic
    df['akaike_weight'] = np.exp(-0.5 * df['delta_AIC'])
    df['akaike_weight'] = df['akaike_weight'] / df['akaike_weight'].sum()
    
    # Calculate BIC weights
    min_bic = df['BIC'].min()
    df['delta_BIC'] = df['BIC'] - min_bic
    df['bic_weight'] = np.exp(-0.5 * df['delta_BIC'])
    df['bic_weight'] = df['bic_weight'] / df['bic_weight'].sum()
    
    # Sort by AIC (best fit first)
    df = df.sort_values('AIC').reset_index(drop=True)
    
    # Round numerical values for display
    df['log_likelihood'] = df['log_likelihood'].round(3)
    df['AIC'] = df['AIC'].round(3)
    df['BIC'] = df['BIC'].round(3)
    df['akaike_weight'] = df['akaike_weight'].round(4)
    df['bic_weight'] = df['bic_weight'].round(4)
    
    return df[['distribution', 'parameters', 'log_likelihood', 'AIC', 'BIC', 
               'akaike_weight', 'bic_weight']]

## Fractal dimention

In [ ]:


def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees) in kilometers
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

def calculate_trajectory_length(df):
    """Calculate total trajectory length using haversine distance"""
    total_length = 0
    for i in range(1, len(df)):
        dist = haversine_distance(
            df.iloc[i-1]['latitude'], df.iloc[i-1]['longitude'],
            df.iloc[i]['latitude'], df.iloc[i]['longitude']
        )
        total_length += dist
    return total_length

def divider_method_fractal_dimension(df, scales=None):
    """
    Calculate fractal dimension using the divider method (ruler method)
    This method measures the trajectory length at different scales
    """
    if scales is None:
        # Generate logarithmically spaced scales
        min_scale = 0.1  # km
        max_scale = 10.0  # km
        scales = np.logspace(np.log10(min_scale), np.log10(max_scale), 20)
    
    lengths = []
    
    for scale in scales:
        # Resample trajectory at given scale intervals
        cumulative_dist = 0
        simplified_points = [0]  # Start with first point
        
        for i in range(1, len(df)):
            dist = haversine_distance(
                df.iloc[simplified_points[-1]]['latitude'], df.iloc[simplified_points[-1]]['longitude'],
                df.iloc[i]['latitude'], df.iloc[i]['longitude']
            )
            cumulative_dist += dist
            
            if cumulative_dist >= scale:
                simplified_points.append(i)
                cumulative_dist = 0
        
        # Calculate length of simplified trajectory
        length = 0.000000001
        for i in range(1, len(simplified_points)):
            idx1, idx2 = simplified_points[i-1], simplified_points[i]
            dist = haversine_distance(
                df.iloc[idx1]['latitude'], df.iloc[idx1]['longitude'],
                df.iloc[idx2]['latitude'], df.iloc[idx2]['longitude']
            )
            length += dist
        
        lengths.append(length)
    
    # Fit power law: L(δ) = C * δ^(1-D)
    # Taking log: log(L) = log(C) + (1-D) * log(δ)
    log_scales = np.log10(scales)
    log_lengths = np.log10(lengths)
    
    # Linear regression
    reg = LinearRegression()
    reg.fit(log_scales.reshape(-1, 1), log_lengths)
    slope = reg.coef_[0]
    
    # Fractal dimension D = 1 - slope
    fractal_dimension = 1 - slope
    
    return fractal_dimension, scales, lengths, slope

# Example usage with your data
def analyze_ship_trajectory():
    # Your trajectory data
    data = {
        'latitude': [33.505038, 33.506835, 33.500507, 33.503695, 33.513497],
        'longitude': [127.450177, 127.393068, 127.332849, 127.273373, 127.213807]
    }
    
    df = pd.DataFrame(data)
    
    print("Ship Trajectory Fractal Dimension Analysis")
    print("=" * 50)
    
    # Calculate fractal dimensions using both methods
    divider_results = divider_method_fractal_dimension(df)
    # box_results = box_counting_fractal_dimension(df)
    
    d_dim, _, _, _ = divider_results
    # b_dim, _, _, _ = box_results
    
    print(f"Divider Method Fractal Dimension: {d_dim:.3f}")
    # print(f"Box Counting Fractal Dimension: {b_dim:.3f}")
    
    # Additional trajectory metrics
    total_length = calculate_trajectory_length(df)
    straight_dist = haversine_distance(
        df.iloc[0]['latitude'], df.iloc[0]['longitude'],
        df.iloc[-1]['latitude'], df.iloc[-1]['longitude']
    )
    
    print(f"Total trajectory length: {total_length:.2f} km")
    print(f"Straight-line distance: {straight_dist:.2f} km")
    print(f"Sinuosity ratio: {total_length/straight_dist:.2f}")
    
    # Plot results
    # plot_fractal_analysis(df, divider_results, box_results)
    
    return df, divider_results



## Code

In [264]:
file = "../../data/FishingKoreaAIS_sampled_new/len_388_mmsi_440716900_eta_val_2024-02-08 15:00:00_dest_JEJU.csv"
len_match = re.search(r'len_(\d+)_mmsi_(\d+)', file)
if len_match:
    length = int(len_match.group(1))
    mmsi = len_match.group(2)
    print(length, mmsi)
df_original = pd.read_csv(file, index_col=0)
df_original[cols.Sampled_Date] = pd.to_datetime(df_original[cols.Sampled_Date], errors="coerce")

df = df_original.copy()
df['target_id'] = mmsi  
# df

388 440716900


In [ ]:
n_to_predict = 20
df = df_original.copy()
df = df.set_index(cols.Sampled_Date)
states_df, covariances, df = run_aekf_example(df, n_to_predict)

df_sequences = get_trajectory_sequences(df)
corrected_df_sequences = get_trajectory_sequences(states_df)

fig = plot_plotly_trajectory_groups([corrected_df_sequences, df_sequences], group_names=["AEKF", "Initial trajectory"])
fig.show()
# fig.write_html(f"../../results/{file.split("/")[-1][:-4]}_n_to_predict_{n_to_predict}.html")

In [ ]:
# max_window = 20
# min_window = 10
window = 20
step = 5
for corrected_df_sequence in corrected_df_sequences:
    for i in range(int(len(corrected_df_sequence) / step)):
        current_segment = corrected_df_sequence[step*i:window+step*i]
        if len(current_segment) < window:
            continue
        
        pseudo_area = (current_segment[cols.Latitude].max() - current_segment[cols.Latitude].min()) * (current_segment[cols.Longitude].max() - current_segment[cols.Longitude].min())
        # print(pseudo_area)
        if pseudo_area < 0.002:
            continue
        
        original_traj_segment = df[(df.index >= current_segment.index.min()) & (df.index <= current_segment.index.max())]
        
        # if i == 11:
        #     print(current_segment)
        #     original_traj_segment_sequences = get_trajectory_sequences(original_traj_segment)
        #     fig = plot_plotly_trajectory_groups([[current_segment], original_traj_segment_sequences], group_names=["AEKF", "Initial trajectory"])
        #     fig.show()
            
        divider_results = divider_method_fractal_dimension(current_segment)
        d_dim, _, _, _ = divider_results
        print(f"Divider Method Fractal Dimension for i={i}: {d_dim:.3f}")

        
        if d_dim > 1.07 and d_dim < 2:
            turning_angles = calculate_turning_angles(current_segment)
            # print(turning_angles)
            
            results = fit_circular_distributions(turning_angles)
            # results = evaluate_circular_distributions(turning_angles)
            print(results)
            
            divider_results = divider_method_fractal_dimension(current_segment)
            d_dim, _, _, _ = divider_results
            print(f"Divider Method Fractal Dimension: {d_dim:.3f}")
            # # Print results
            # for dist_name, info in results.items():
            #     print(f"\nDistribution: {dist_name}")
            #     print(f"Log-Likelihood: {info['log_likelihood']:.4f}")
            #     print(f"Parameters: {info['parameters']}")
        
            fig = plot_circular_distribution(turning_angles, bins=90)
            fig.show()
            
            
            # break
            print(current_segment)
            # print(original_traj_segment)
            original_traj_segment_sequences = get_trajectory_sequences(original_traj_segment)
            fig = plot_plotly_trajectory_groups([[current_segment], original_traj_segment_sequences], group_names=["AEKF", "Initial trajectory"])
            fig.show()
            # break
        # print(i, len(original_traj))
        
        # if i == 4:
        #     print(current_segment)
        #     print(original_traj)
        